In [23]:
from pymongo import MongoClient
import pandas as pd
from pandas.io.json import json_normalize
import pickle

In [ ]:
import pandas as pd
import pandas
from itertools import chain
import numpy as np

from bson import json_util, ObjectId
from pandas.io.json import json_normalize
import json



In [2]:
client = MongoClient()
client.list_database_names()

['admin', 'books', 'comedy_covid', 'config', 'local', 'outings', 'ted']

In [4]:
db = client.ted
db.list_collection_names()

['ted_stats', 'ted_transcripts', 'ted_comments']

In [28]:
#cursor = db.ted_comments.find()
#list(cursor)

In [6]:
cursor = db.ted_comments.find({}, {'_id':0, 'items.snippet':1})

In [9]:
df =  pd.DataFrame(list(cursor))

In [15]:
df.head()

,items
0,"[{'snippet': {'videoId': 'YgAuFqEs6yk', 'topLe..."
1,"[{'snippet': {'videoId': 'bNmRr-BYnxA', 'topLe..."
2,"[{'snippet': {'videoId': 'FVUkKKc3Vvk', 'topLe..."
3,"[{'snippet': {'videoId': '8bj0GR34XWc', 'topLe..."
4,"[{'snippet': {'videoId': 'eaCrsBtiYA4', 'topLe..."


In [184]:
#df['items'][0]

In [13]:
new_df = pd.concat([pd.DataFrame(json_normalize(x)) for x in df['items']]) #,ignore_index=True

/Users/elena/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [14]:
new_df.columns

Index(['snippet.canReply', 'snippet.isPublic', 'snippet.topLevelComment.etag',
       'snippet.topLevelComment.id', 'snippet.topLevelComment.kind',
       'snippet.topLevelComment.snippet.authorChannelId.value',
       'snippet.topLevelComment.snippet.authorChannelUrl',
       'snippet.topLevelComment.snippet.authorDisplayName',
       'snippet.topLevelComment.snippet.authorProfileImageUrl',
       'snippet.topLevelComment.snippet.canRate',
       'snippet.topLevelComment.snippet.likeCount',
       'snippet.topLevelComment.snippet.publishedAt',
       'snippet.topLevelComment.snippet.textDisplay',
       'snippet.topLevelComment.snippet.textOriginal',
       'snippet.topLevelComment.snippet.updatedAt',
       'snippet.topLevelComment.snippet.videoId',
       'snippet.topLevelComment.snippet.viewerRating',
       'snippet.totalReplyCount', 'snippet.videoId'],
      dtype='object')

In [16]:
new_df.shape

(43303, 19)

In [17]:
new_df_text2 = pd.DataFrame(new_df.groupby(['snippet.videoId'])
                            ['snippet.topLevelComment.snippet.textDisplay'].apply(
                                lambda x: "{%s}" % ', '.join(x)))

In [20]:
new_df_text2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2193 entries, -2Dj9M71JAc to zwpiI18TBdE
Data columns (total 1 columns):
snippet.topLevelComment.snippet.textDisplay    2193 non-null object
dtypes: object(1)
memory usage: 34.3+ KB


In [34]:
new_df_text2.sample(2)

,video_id,comments
79,1XL8I9-IqRY,"{I&#39;m sorry, but ignorants are a reality, t..."
1724,nLecuOcB3dA,{парк под землей с солнечным светом.... интере...


In [33]:
new_df_text2.reset_index(inplace=True)
new_df_text2.rename(columns={'snippet.topLevelComment.snippet.textDisplay':'comments', 
                             'snippet.videoId': 'video_id'}, inplace=True)

In [35]:
with open('ted_video_comments_2193.pkl', 'wb') as picklefile:
    pickle.dump(new_df_text2, picklefile)